In [1]:
import torch
import numpy as np
from transformers import (
    Trainer,
    TrainingArguments,
    AutoModelForSequenceClassification,
    AutoTokenizer
)

In [2]:
# load dataset
from datasets import load_dataset

klue_tc_train = load_dataset("klue", "ynat", split="train")
klue_tc_eval = load_dataset("klue", "ynat", split="validation")
klue_tc_train

README.md:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/4.17M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/847k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/45678 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/9107 [00:00<?, ? examples/s]

Dataset({
    features: ['guid', 'title', 'label', 'url', 'date'],
    num_rows: 45678
})

In [5]:
klue_tc_train = klue_tc_train.remove_columns(['guid', 'url', 'date'])
klue_tc_eval = klue_tc_eval.remove_columns(['guid', 'url', 'date'])
print(klue_tc_train)
# Dataset({
#     features: ['title', 'label'],
#     num_rows: 45678
# })

# dataset 일부만 사용
train_dataset = klue_tc_train.train_test_split(test_size=10000, shuffle=True, seed=42)['test']
dataset = klue_tc_eval.train_test_split(test_size=1000, shuffle=True, seed=42)
test_dataset = dataset['test']
valid_dataset = dataset['train'].train_test_split(test_size=1000, shuffle=True, seed=42)['test']

Dataset({
    features: ['title', 'label'],
    num_rows: 45678
})


In [6]:
def tokenize_function(examples):
    return tokenizer(examples["title"], padding="max_length", truncation=True)

model_id = "klue/roberta-base"
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=len(train_dataset.features['label'].names))
tokenizer = AutoTokenizer.from_pretrained(model_id)

train_dataset = train_dataset.map(tokenize_function, batched=True)
valid_dataset = valid_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./results", # 결과를 저장할 디렉토리
    num_train_epochs=3,     # 학습 에포크 수
    per_device_train_batch_size=8,  # 학습 배치 사이즈
    per_device_eval_batch_size=8,   # 평가 배치 사이즈
    eval_strategy="epoch",    # 에포크가 끝날 때 마다 평가 수행
    learning_rate=5e-5,
    push_to_hub=False
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # logits: 모델의 출력
    #   - 형태 : [batch_size, num_classes]
    #   - 예시 : [[2.1, -0.5, 1.8], [0.2, 3.1, -1.0]] (배치 크기 2, 클래스 3개인 경우)
    # labels: 실제 레이블
    #   - 형태 : [batch_size]
    #   - 예시 : [0, 1] (첫 번째 샘플은 클래스 0, 두 번째 샘플은 클래스 1)

    predictions = np.argmax(logits, axis=-1)
    # 각 샘플에 대해 가장 높은 점수를 가진 클래스의 인덱스를 반환
    # 첫 번째 샘플 [2.1, -0.5, 1.8] → 0 (2.1이 가장 큼)
    # 두 번째 샘플 [0.2, 3.1, -1.0] → 1 (3.1이 가장 큼)
    # 결과: predictions = [0, 1]
    # axis = -1 : 마지막 차원을 지정. 2차원 배열에서는 각 행(샘플)에 대해 가로 방향(클래스 방향)으로 최댓값의 위치(인덱스)를 찾음
    #   - 3차원 배열이라면 axis=1 은 두 번째 차원을 지정함

    return {"accuracy": (predictions == labels).mean()}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

trainer.evaluate(test_dataset)


/var/folders/x2/l70rpjrd0m5ffjnv7btdj1m40000gn/T/ipykernel_1948/1500351807.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


KeyboardInterrupt: 